In [1]:
#Importing the dataset and libraries
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
import plotly.graph_objects as go
from tensorflow.keras import regularizers
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping


In [2]:
#Checking for GPU availability

if tf.test.gpu_device_name():
    print('GPU device found: {}'.format(tf.test.gpu_device_name()))
    device = '/device:GPU:0'
else:
    print('No GPU device found. Using CPU.')
    device = '/device:CPU:0'

GPU device found: /device:GPU:0


In [3]:
(x_train, y_train),(x_test, y_test) = mnist.load_data() #Splitting data into test and train  

x_train = x_train.reshape(-1, 784) / 255.0              #Flattening the (28X28) data to (784x1)
x_test = x_test.reshape(-1, 784) / 255.0                #and normalizing the greyscale intensities


x_validation, x_train = x_train[:5000], x_train[5000:]  #5000 data points for validation
y_validation, y_train = y_train[:5000], y_train[5000:]

y_train = tf.keras.utils.to_categorical(y_train)
y_validation = tf.keras.utils.to_categorical(y_validation)
y_test = tf.keras.utils.to_categorical(y_test)

11490434/11490434 [==============================] - 1s 0us/step


In [4]:
x_train.shape, x_validation.shape, x_test.shape, y_train.shape, y_validation.shape, y_test.shape

((55000, 784), (5000, 784), (10000, 784), (55000, 10), (5000, 10), (10000, 10))

In [5]:
#Neural Network with 3 layers, relu as activation for hidden layers
#with softmax as activation for the output layer. Regularizer used: l2 and C: 0.001
#dropout set to 50%
model = Sequential()
model.add(Dense(500, activation='relu', kernel_regularizer=l2(0.001), input_shape=(784,)))
model.add(Dropout(0.5))
model.add(Dense(500, activation='relu', kernel_regularizer=l2(0.001)))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

In [6]:
#Compiling the model with Adam Optimizer with categorical cross entropy as the loss function
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [7]:
#Setting batch size = 128 and epochs to 250
batch_size = 128
epochs = 250

In [8]:
with tf.device(device):
    
    loss_values = []
    error_values = []

    #each iteration of epochs
    for epoch in range(epochs):
        
        #each iteration of batch
        for batch_start in range(0, len(x_train), batch_size):
            batch_end = batch_start + batch_size
            x_batch = x_train[batch_start:batch_end]
            y_batch = y_train[batch_start:batch_end]

            model.train_on_batch(x_batch, y_batch)

        #calculating the loss and accuracy
        loss, accuracy = model.evaluate(x_validation, y_validation, batch_size=batch_size, verbose=0)

        #calculating the error
        error = 1 - accuracy
        loss_values.append(loss)
        error_values.append(error)
        
        print(f"Epoch {epoch+1}/{epochs} - Loss: {loss:.4f} - Error: {error:.4f}")

Epoch 1/250 - Loss: 0.5056 - Error: 0.0458
Epoch 2/250 - Loss: 0.3687 - Error: 0.0422
Epoch 3/250 - Loss: 0.3023 - Error: 0.0350
Epoch 4/250 - Loss: 0.3062 - Error: 0.0398
Epoch 5/250 - Loss: 0.2927 - Error: 0.0354
Epoch 6/250 - Loss: 0.2793 - Error: 0.0332
Epoch 7/250 - Loss: 0.2734 - Error: 0.0282
Epoch 8/250 - Loss: 0.2783 - Error: 0.0328
Epoch 9/250 - Loss: 0.2771 - Error: 0.0306
Epoch 10/250 - Loss: 0.2796 - Error: 0.0346
Epoch 11/250 - Loss: 0.2904 - Error: 0.0354
Epoch 12/250 - Loss: 0.2737 - Error: 0.0290
Epoch 13/250 - Loss: 0.2623 - Error: 0.0260
Epoch 14/250 - Loss: 0.2622 - Error: 0.0300
Epoch 15/250 - Loss: 0.2638 - Error: 0.0294
Epoch 16/250 - Loss: 0.2597 - Error: 0.0292
Epoch 17/250 - Loss: 0.2606 - Error: 0.0280
Epoch 18/250 - Loss: 0.2636 - Error: 0.0338
Epoch 19/250 - Loss: 0.2523 - Error: 0.0266
Epoch 20/250 - Loss: 0.2710 - Error: 0.0336
Epoch 21/250 - Loss: 0.2621 - Error: 0.0296
Epoch 22/250 - Loss: 0.2656 - Error: 0.0296
Epoch 23/250 - Loss: 0.2586 - Error: 0.02

In [9]:

with tf.device(device):
    #Early Stopping
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, restore_best_weights=True)

    history = model.fit(
        x_train,
        y_train,
        batch_size=batch_size,
        epochs=epochs,
        validation_data=(x_test, y_test),
        callbacks=[early_stopping]
    )

Epoch 1/250
430/430 [==============================] - 3s 6ms/step - loss: 0.2698 - accuracy: 0.9584 - val_loss: 0.2178 - val_accuracy: 0.9746
Epoch 2/250
430/430 [==============================] - 2s 5ms/step - loss: 0.2727 - accuracy: 0.9573 - val_loss: 0.2103 - val_accuracy: 0.9764
Epoch 3/250
430/430 [==============================] - 2s 5ms/step - loss: 0.2757 - accuracy: 0.9566 - val_loss: 0.2153 - val_accuracy: 0.9736
Epoch 4/250
430/430 [==============================] - 2s 5ms/step - loss: 0.2712 - accuracy: 0.9571 - val_loss: 0.2087 - val_accuracy: 0.9755
Epoch 5/250
430/430 [==============================] - 2s 5ms/step - loss: 0.2724 - accuracy: 0.9567 - val_loss: 0.2112 - val_accuracy: 0.9754
Epoch 6/250
430/430 [==============================] - 2s 4ms/step - loss: 0.2747 - accuracy: 0.9568 - val_loss: 0.2205 - val_accuracy: 0.9738
Epoch 7/250
430/430 [==============================] - 3s 6ms/step - loss: 0.2737 - accuracy: 0.9560 - val_loss: 0.2156 - val_accuracy: 0.9739

In [10]:
#Accessing the different metrics and saving it to plot the graph
loss_values = history.history['loss']
val_loss_values = history.history['val_loss']
acc_values = history.history['accuracy']
val_acc_values = history.history['val_accuracy']

In [25]:
print(val_error_values)

[0.025399982929229736, 0.023599982261657715, 0.02640002965927124, 0.024500012397766113, 0.02460002899169922, 0.02619999647140503, 0.026099979877471924, 0.028299987316131592, 0.025399982929229736, 0.025600016117095947, 0.024399995803833008, 0.02369999885559082, 0.029600024223327637, 0.027300000190734863]


In [26]:
epochs_range = range(1, len(loss_values) + 1)

# Create the loss plot
loss_plot = go.Scatter(x=list(epochs_range), y=loss_values, mode='lines', name='Training Loss')
val_loss_plot = go.Scatter(x=list(epochs_range), y=val_loss_values, mode='lines', name='Validation Loss')
layout_loss = go.Layout(title='Cross-Entropy Loss', xaxis=dict(title='Epoch'), yaxis=dict(title='Loss'))
fig_loss = go.Figure(data=[loss_plot, val_loss_plot], layout=layout_loss)
fig_loss.show()

#Creating the classification error plot
val_error_values = [1 - acc for acc in val_acc_values]
err_plot = go.Scatter(x=list(epochs_range), y=error_values, mode='lines', name='Training Error')
val_error_plot = go.Scatter(x=list(epochs_range), y=val_error_values, mode='lines', name='Validation Error')
layout_acc = go.Layout(title='Error', xaxis=dict(title='Epoch'), yaxis=dict(title='Error'))
fig_acc = go.Figure(data=[err_plot, val_error_plot], layout=layout_acc)
fig_acc.show()


# Create the accuracy plot
acc_plot = go.Scatter(x=list(epochs_range), y=acc_values, mode='lines', name='Training Accuracy')
val_acc_plot = go.Scatter(x=list(epochs_range), y=val_acc_values, mode='lines', name='Validation Accuracy')
layout_acc = go.Layout(title='Accuracy', xaxis=dict(title='Epoch'), yaxis=dict(title='Accuracy'))
fig_acc = go.Figure(data=[acc_plot, val_acc_plot], layout=layout_acc)
fig_acc.show()


In [16]:
print(model.evaluate(x_test, y_test))         #Evaluating the model's loss and accuracy

313/313 [==============================] - 1s 4ms/step - loss: 0.2087 - accuracy: 0.9755
[0.20873022079467773, 0.9754999876022339]
